<a href="https://colab.research.google.com/github/ismailasy/Deep_IOT_Network_Anomaly_Detection/blob/main/Classification_multiclasse_6_Classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pour cette section nous allons essayer de faire une classification multiclasse en 5 classes:


1.   MITM(attaque man in the midddle)
2.   Injection 
3.   Malware 
4.   DDos(Deni de service distribué)
5.   Vol d'informations






In [1]:
# Pour commencer nous allons importzr le jeu de données Iot dataset
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/Datasets/Selected dataset for ML and DL/DNN-EdgeIIoT-dataset.csv', low_memory=False)

In [2]:
# Affichage des types d'attaques 
print(df['Attack_type'].value_counts())

Normal                   1615643
DDoS_UDP                  121568
DDoS_ICMP                 116436
SQL_injection              51203
Password                   50153
Vulnerability_scanner      50110
DDoS_TCP                   50062
DDoS_HTTP                  49911
Uploading                  37634
Backdoor                   24862
Port_Scanning              22564
XSS                        15915
Ransomware                 10925
MITM                        1214
Fingerprinting              1001
Name: Attack_type, dtype: int64


Pour commencer nous allons regroupper les categories d'attaques de meme type sur une seule classe

*   MITM 
*   Injection : XSS,SQL_injection,Uploading
*   Malware : Backdoor,Ransomware,Password 
*   DDoS(DDoS_UDP,DDoS_ICMP,DDoS_TCP,DDoS_HTTP )
*   Vol d'informations :Port_Scanning,Vulnerability_scanner,Fingerprinting              

